# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup

import hvplot.pandas
import pandas as pd
import requests

# Import API key

from config import api_key2

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame

city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data

city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mitu,1.1983,-70.1733,23.23,93,15,0.93,CO,1697679989
1,1,albany,42.6001,-73.9662,11.24,89,79,0.97,US,1697680228
2,2,anna regina,7.2644,-58.5077,27.86,78,59,3.33,GY,1697680268
3,3,kuressaare,58.2481,22.5039,0.53,93,20,1.54,EE,1697680269
4,4,charleston,32.7766,-79.9309,14.45,89,0,2.06,US,1697680140


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

Humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
weather = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1200,
    frame_height = 600,
    size = "Humidity",
    color="City",
)

# Display the map

weather

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values

city_data_df = city_data_df.dropna()

# Display sample data

city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
48,48,lompoc,34.6391,-120.4579,24.45,94,0,2.06,US,1697680273
58,58,remire-montjoly,4.9167,-52.2667,24.02,100,0,0.00,GF,1697680274
87,87,san patricio,28.0170,-97.5169,24.91,65,0,4.03,US,1697680277
123,123,naze,28.3667,129.4833,23.51,51,0,1.27,JP,1697680281
130,130,acarau,-2.8856,-40.1200,25.47,81,0,3.82,BR,1697680282
147,147,upington,-28.4478,21.2561,21.21,28,0,2.06,ZA,1697680284
213,213,jahazpur,25.6167,75.2833,22.63,54,0,1.90,IN,1697680291
300,300,mirpur sakro,24.5474,67.6277,26.55,52,0,3.12,PK,1697680300
343,343,tindouf,27.6711,-8.1474,24.25,22,0,3.60,DZ,1697680305
407,407,tura,25.5198,90.2201,23.37,77,0,0.52,IN,1697680312


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = city_data_df[{"City": city,
                        "Country": country,
                        "Lat": lat,
                        "Lng": lng,
                        "Humidity": humidity}]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = ""

# Display sample data

hotel_df

C:\Users\baseb\AppData\Local\Temp\ipykernel_29576\2894099525.py:8: FutureWarning: Passing a dict as an indexer is deprecated and will raise in a future version. Use a list instead.
  hotel_df = city_data_df[{"City": city,
C:\Users\baseb\AppData\Local\Temp\ipykernel_29576\2894099525.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
48,lompoc,US,34.6391,-120.4579,94,
58,remire-montjoly,GF,4.9167,-52.2667,100,
87,san patricio,US,28.0170,-97.5169,65,
123,naze,JP,28.3667,129.4833,51,
130,acarau,BR,-2.8856,-40.1200,81,
147,upington,ZA,-28.4478,21.2561,28,
213,jahazpur,IN,25.6167,75.2833,54,
300,mirpur sakro,PK,24.5474,67.6277,52,
343,tindouf,DZ,27.6711,-8.1474,22,
407,tura,IN,25.5198,90.2201,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":api_key2
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data

hotel_df

Starting hotel search
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
remire-montjoly - nearest hotel: Complexe Belova
san patricio - nearest hotel: No hotel found
naze - nearest hotel: ビジネス旅館畠山
acarau - nearest hotel: No hotel found
upington - nearest hotel: Die Eiland Holiday Ressort
jahazpur - nearest hotel: No hotel found
mirpur sakro - nearest hotel: No hotel found
tindouf - nearest hotel: محمد بوسبي
tura - nearest hotel: No hotel found
medina - nearest hotel: The Oberoi Madina
leteri - nearest hotel: No hotel found
shahzadpur - nearest hotel: No hotel found
noumea - nearest hotel: Hotel Gondwana
ormara - nearest hotel: No hotel found
kooringal - nearest hotel: The Astor
pahrump - nearest hotel: Holiday Inn Express & Suites
belem - nearest hotel: Hotel Portas da Amazônia
palmas - nearest hotel: Hotel Triângulo Mineiro
chandla - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
48,lompoc,US,34.6391,-120.4579,94,Embassy Suites by Hilton Lompoc Central Coast
58,remire-montjoly,GF,4.9167,-52.2667,100,Complexe Belova
87,san patricio,US,28.0170,-97.5169,65,No hotel found
123,naze,JP,28.3667,129.4833,51,ビジネス旅館畠山
130,acarau,BR,-2.8856,-40.1200,81,No hotel found
147,upington,ZA,-28.4478,21.2561,28,Die Eiland Holiday Ressort
213,jahazpur,IN,25.6167,75.2833,54,No hotel found
300,mirpur sakro,PK,24.5474,67.6277,52,No hotel found
343,tindouf,DZ,27.6711,-8.1474,22,محمد بوسبي
407,tura,IN,25.5198,90.2201,77,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
# Configure the map plot

hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1200,
    frame_height = 600,
    size = "Humidity",
    color="City",
    hover_cols=["Hotel Name", "Country"])

# Display the map

hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)